#Предсказание рейтинга ресторана по версии TripAdvisor
Собственно это каггловское соревнование со следующими вводными:

В этом соревновании вам будет предложен датасет, содержащий сведения о ресторанах. С помощью имеющего в вашем распоряжении кода, вам необходимо создать модель, использующую алгоритм RandomForestRegression, которая будет прогнозировать рейтинг ресторана по версии TripAdvidor.

Основная цель: качественно очистить датасет, подобрать подходящие значения для заполнения пропусков и создать новые признаки на основе той информации, которую вы сможете извлечь из имеющихся в вашем распоряжении данных.

Условия соревнования: Все участники должны использовать один и тот же алгоритм с параметрами, заданными по умолчанию. Разрешено использовать внешние данные.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
RANDOM_SEED = 42

In [ ]:
!pip freeze > requirements.txt

In [ ]:
# при загрузке использую расширение .xls, так как при сохранении файлов на компьютер, они автоматически сохраняются
# в том формате.
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
# А теперь неожиданно!!! - помечаем датафреймы и объединяем в один (предварительно создав колонку Rating в df_test)
df_train['sample'] = 1 
df_test['sample'] = 0 
df_test['Rating'] = 0 

df = df_test.append(df_train, sort=False).reset_index(drop=True)

dffin = pd.DataFrame()
dffin['Restaurant_id'] = df['Restaurant_id']

#df = df.drop(['Name'], axis=1)

In [ ]:
# Посмотрим, что получилось
df.head(3)

In [ ]:
# ПЕРВЫМ ДЕЛОМ РАЗБЕРЕМСЯ С ПРОПУСКАМИ В ДАННЫХ

In [ ]:
# Чтобы заполнить недостающие данные в столбце Cuisine Stile, самые популярные виды кухонь в каждом городе
# сделаем датафрейм с городами и самыми популярными там кухнями
df2 = df.dropna(subset=['Cuisine Style'], inplace=False)
df2 = df2.groupby('City')['Cuisine Style'].sum().apply(lambda x:x.replace('[','').replace(']','').replace('\'','').split(', '))

# Так как среднее количество кухонь на 1 ресторан = 2.6, то найдем три самые популярные кухни для каждого города
mostpop=[]
for i in range(len(df2)):
    pop = []
    trt = pd.Series(df2[i]).value_counts() # нашли самые популярные кухни
    pop.append(pd.DataFrame(trt).index[0])
    pop.append(pd.DataFrame(trt).index[1])
    pop.append(pd.DataFrame(trt).index[2])
    mostpop.append(pop) # сунули их в список

mostpop = pd.Series(mostpop)

cit = pd.DataFrame(df2.index)

cit_cus = pd.concat([cit,mostpop], axis=1)
cit_cus.columns = ['City', 'Most pop cuisine']
cit_cus['Capital'] = 0

In [ ]:
# Че получилось:
cit_cus.head()

In [ ]:
# Очистим столбец Cuisine Stile и переведем их в список
df['Cuisine Style'] = df['Cuisine Style'].loc[df['Cuisine Style'].isna() == False].apply(lambda x:x.replace('[','').replace(']','').replace('\'','').split(', '))

# Соединим две таблицы
df = df.merge(cit_cus, on='City', how = 'left')

# Заполним nan-ы из Cuisine Style данными из Most pop cuisine
df['Cuisine Style'] = df['Cuisine Style'].fillna(df['Most pop cuisine'])

# Ну и удалим 'Most pop cuisine'
df = df.drop(['Most pop cuisine'], axis=1)

In [ ]:
df.info()

Теперь колонка Cuisine Style заполнена

In [ ]:
# запилим список со столицами государств
capitals = ['London', 'Paris', 'Madrid', 'Berlin', 'Rome', 'Prague', 'Lisbon', 'Vienna', 'Amsterdam', 'Brussels', 
            'Stockholm', 'Budapest', 'Warsaw', 'Dublin', 'Copenhagen', 'Athens', 'Edinburgh', 'Oslo', 'Helsinki', 
            'Bratislava', 'Luxembourg', 'Ljubljana']

In [ ]:
# отметим в датафрейме столицы - 1, остальные города - 0
df['Capital'].loc[df['City'].isin(capitals)] = 1

In [ ]:
df.head(3)

Теперь разберемся с колонкой Price

In [ ]:
df['Price Range'].unique()

In [ ]:
# Первым делом переведем существующие занчения в числовые аналоги
# Для того запилим датафрейм
dfprice = pd.DataFrame({'Price Range': ['$', '$$ - $$$', 
                                        '$$$$'], 'Price New': [1,2, 3]})
df = df.merge(dfprice, on='Price Range', how = 'left')

# И удалим столбец 'Price Range', чтоб он нас более не смущал
df = df.drop(['Price Range'], axis=1)

In [ ]:
df['Price New'].value_counts()

Найдем для каждого города наиболее верояную ценовую политику ресторанов

In [ ]:
dict_city_price={}

for city in df['City'].unique():
    price = df[df['City']==city]['Price New'].value_counts().index[0]
    dict_city_price[city]=price

In [ ]:
dict_city_price

Как видим, для всех городов наиболее частая ценовая категория - 2. Поэтому просто заполним пропуски в данных двойкой

In [ ]:
df['Price New'] = df['Price New'].fillna(2)

In [ ]:
df.info()

In [ ]:
# Найдем, колько дней прошло с момента последнего отзыва

import re

df['Reviews'] = df['Reviews'].apply(lambda x: str([[], []]) if type(x) == float else x) 
df['Reviews'] = df['Reviews'].apply(lambda x: str(x) if type(x) == list else x)

# Вытащим даты отзывов из столбца
res = []
for i in df['Reviews']:
    res.append(re.findall(r'(\d\d/\d\d/\d\d\d\d)', i))
reviews = pd.DataFrame(res)
reviews[0] = pd.to_datetime(reviews[0])
reviews[1] = pd.to_datetime(reviews[1])

In [ ]:

import datetime as dt
from datetime import date, timedelta

today = pd.to_datetime(date.today().strftime('%Y-%m-%d'))

fromtoday = []

# Как видим, в столбце 0 даты более поздние, поэтому будем искать разницу именно с ними
for i in range(len(reviews)):
    fromtoday.append(today - reviews.loc[i, 0])

# Создадим столбец под эти бесценные данные    
df['fromtoday'] = pd.Series(fromtoday)
df['fromtoday'] = df['fromtoday'].apply(lambda x: x.days if (type(x) != int) else 0)
df['fromtoday'] = df['fromtoday'].fillna(0)

# И, раз пошла такая пьянка, найдем разницу между двумя отзывами

df['delta_days'] = reviews[0] - reviews[1]
df['delta_days'] = df['delta_days'].apply(lambda x: x.days if type(x) != float else 0)
df['delta_days'] = df['delta_days'].fillna(0)

In [ ]:
# Сделаем столбец под id ресторана

df['id'] = df['Restaurant_id'].apply(lambda x: int(x[3:]))

In [ ]:
# Добавим некоторые экономические показатели для каждого города

GDP = {'London': 41030, 'Paris': 41760, 'Madrid': 29961, 'Barcelona': 29961, 'Berlin': 46563, 'Milan': 32946,
       'Rome': 32946, 'Prague': 23213, 'Lisbon': 23030, 'Vienna': 50022, 'Amsterdam': 52367, 'Brussels': 45175, 
       'Hamburg': 46563, 'Munich': 46563, 'Lyon': 41760, 'Stockholm': 51241, 'Budapest': 17463, 'Warsaw': 14901,
       'Dublin': 77771, 'Copenhagen': 59795, 'Athens': 19974, 'Edinburgh': 42500, 'Zurich': 83716, 'Oporto': 23030,
       'Geneva': 83716, 'Krakow': 14901, 'Oslo': 77957, 'Helsinki': 48868, 'Bratislava': 19547, 'Luxembourg': 113196,
       'Ljubljana': 26170}


CityPop = {'Paris':2140526, 'Stockholm': 1632798, 'London': 9126366, 'Berlin':3748148, 'Munich':1456039, 'Oporto': 2400000,
       'Milan':1405879, 'Bratislava':434926, 'Vienna':1899055, 'Rome':2857321, 'Barcelona':1620343, 'Madrid':3223334,
       'Dublin':1361000, 'Brussels':1211035, 'Zurich':1383000, 'Warsaw':1802237, 'Budapest':1749734, 'Copenhagen':1334000,
       'Amsterdam':1140000, 'Lyon':513275, 'Hamburg':1930996, 'Lisbon':2927000, 'Prague':1308632, 'Oslo':1041377,
       'Helsinki':1304851, 'Edinburgh':531000, 'Geneva':201741, 'Ljubljana':292988, 'Athens':3154000,
       'Luxembourg':613894, 'Krakow':762508}

In [ ]:
# Делаем колонку GDP
dfGDP = pd.Series(GDP).reset_index()
dfGDP.columns = ['City', 'GDP']
df = df.merge(dfGDP, on='City', how='left')

# Делаем колонку населения
dfPop = pd.Series(CityPop).reset_index()
dfPop.columns = ['City', 'Population']
df = df.merge(dfPop, on='City', how='left')

# Подсчитываем общее количество ресторанов в каждом городе
dfUnRe = df.groupby('City')['Restaurant_id'].nunique().reset_index()
dfUnRe.columns = ['City', 'Rest Count']
df = df.merge(dfUnRe, on='City', how='left')

# Найдем относительные показатели GDP
df['GDR relativ'] = df['GDP'].apply(lambda x: x/(df['GDP'].max()))

# Найдем относительные показатели населенности
df['Population relativ'] = df['Population'].apply(lambda x: x/(df['Population'].max()))

In [ ]:
df.head(3)

In [ ]:
# Найдем средние значения количества отзывов по городам
dfNoR = df.groupby('City')['Number of Reviews'].mean().reset_index()
dfNoR.columns = ['City', 'Review mean']
df = df.merge(dfNoR, on='City', how='left')

# Теперь заполним недостающие значения в Number of Reviews
#df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Review mean']) - сперва заполнял средним по городам,
# но, как оказалось, при заполнении недостающих занчений единицей, МАЕ снижается
df['Number of Reviews'] = df['Number of Reviews'].fillna(1)

добавим еще несколько фитч:

In [ ]:

# Чисто для прикола нашел в каких городах сколько есть мишленовских звезд у ресторанов. Вдруг, это может отражать
# и общее качество ресторанов в городе?
MichStars = {'Paris':118, 'Stockholm':0, 'London': 66, 'Berlin':16, 'Munich':13, 'Oporto': 0,
       'Milan':15, 'Bratislava':0, 'Vienna':0, 'Rome':15, 'Barcelona':20, 'Madrid':15,
       'Dublin':0, 'Brussels':21, 'Zurich':11, 'Warsaw':0, 'Budapest':0, 'Copenhagen':11,
       'Amsterdam':11, 'Lyon':15, 'Hamburg':11, 'Lisbon':0, 'Prague':0, 'Oslo':0,
       'Helsinki':0, 'Edinburgh':0, 'Geneva':0, 'Ljubljana':0, 'Athens':0,
       'Luxembourg':16, 'Krakow':0}

dfMS = pd.Series(MichStars).reset_index()
dfMS.columns = ['City', 'Mich Stars']
df = df.merge(dfPrice, on='City', how='left')

In [ ]:
# Рэнкинг ресторана, относительно города, в котором этот ресторан расположен
df['Ranking mean'] = df['Ranking']/ df['Rest Count']

# Сколько человек на 1 ресторан в городе
df['Men per Rest'] = df['Population'] / df['Rest Count']

# Рэнкинг ресторана, относительно города и количества человек на ресторан
df['Ranking mean2'] = df['Ranking']/ (df['Rest Count']*df['Men per Rest'])

# Посмотрим, сколько отзывов ресторана приходится на одного человека
df['Review relative'] = df['Number of Reviews'] / df['Men per Rest']

# Найдем Ranking относительнос количества человек на ресторан
df['Ranking relative'] = df['Ranking']/df['Men per Rest']

# Найдем цены, относительно GDP
df['Price relative'] = df['GDR relativ']*df['Price New']

In [ ]:

# Посмотрим, есть ли какие корреляции среди наших переменных. А то может зря я все выше налепил!)

sns.set(font_scale=1)
plt.subplots(figsize=(12, 12))
sns.heatmap(df.corr(), square=True,
              annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu")

In [ ]:
# Итак нам определенно не нужны следующие колонки: Ranking relative, Review relative, Rest Count, Population relativ,
# GDR relativ, Price mean_x

df = df.drop(['Ranking relative','Review relative','Rest Count','Population relativ',
         'GDR relativ', 'Price relative', 'Ranking mean2'],axis=1)

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize=(12, 12))
sns.heatmap(df.corr(), square=True,
              annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu")

In [ ]:
df.info()

In [ ]:
# Создадим думисы для City
df = pd.get_dummies(df, columns=[ 'City',], dummy_na=True)

# Создадим думисы для Price New
df = pd.get_dummies(df, columns=['Price New'], dummy_na=True)

In [ ]:
# А вот с думмисами для Cuisine Style придется немножно посложнее
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
DummCuisin = pd.DataFrame(mlb.fit_transform(df['Cuisine Style']))

In [ ]:
df = pd.concat([df,DummCuisin],axis=1)

In [ ]:
# Удаляем колонки с данными типа object
object_columns = [s for s in df.columns if df[s].dtypes == 'object']

df.drop(object_columns, axis = 1, inplace=True)

In [ ]:
df.head(3)

In [ ]:
df.info()

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

Заметим, что исходные рейтинги ресторанов округляются с шагом 0.5. Поэтому и предсказанные тоже округлим с шагом 0.5

In [ ]:
def round_custom(num):
    return round(num / 0.5) * 0.5

In [ ]:
for i in predict_submission:
    i=round_custom(i)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)